In [1]:
import yfinance as yf
import datetime
import pandas as pd
import re
import time
import numpy as np
import holidays
from datetime import datetime, timedelta
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('all_market_earnings.csv')
df.head()

,id,ticker_id,earnings_date,eps_estimate,reported_eps,surprise,ticker,strike_width,weekly_expiration
0,2,2,2025-06-10 20:00:00,0.00,0.00,0.0000,AVGO,5.0,1
1,3,2,2025-03-05 10:00:00,0.00,0.00,0.0000,AVGO,5.0,1
2,4,2,2024-12-05 21:00:00,0.00,0.00,0.0000,AVGO,5.0,1
3,5,2,2024-08-29 10:00:00,1.20,0.00,0.0000,AVGO,5.0,1
4,6,2,2024-06-12 20:00:00,10.84,10.96,0.0107,AVGO,5.0,1


In [3]:
df = df[df['reported_eps'] != 0].reset_index(drop=True)
df

,id,ticker_id,earnings_date,eps_estimate,reported_eps,surprise,ticker,strike_width,weekly_expiration
0,6,2,2024-06-12 20:00:00,10.84,10.96,0.0107,AVGO,5.0,1
1,7,2,2024-03-07 21:00:00,10.29,10.99,0.0678,AVGO,5.0,1
2,8,2,2023-12-07 21:00:00,10.98,11.06,0.0071,AVGO,5.0,1
3,9,2,2023-08-31 20:00:00,10.42,10.54,0.0112,AVGO,5.0,1
4,10,2,2023-06-01 20:00:00,10.08,10.32,0.0236,AVGO,5.0,1
...,...,...,...,...,...,...,...,...,...
4331,6507,554,2023-08-08 11:00:00,1.32,1.41,0.0719,ZTS,NaN,0
4332,6508,554,2023-05-04 11:00:00,1.26,1.31,0.0389,ZTS,NaN,0
4333,6509,554,2023-02-14 12:00:00,1.05,1.15,0.0990,ZTS,NaN,0
4334,6510,554,2022-11-03 11:00:00,1.24,1.21,-0.0207,ZTS,NaN,0


In [4]:
df['earnings_date'] = pd.to_datetime(df['earnings_date'])

df['date'] = df['earnings_date'].apply(
    lambda x: x.date() - pd.Timedelta(days=1) if x.time() <= pd.Timestamp('13:00:00').time() else x.date()
)

In [5]:
df

,id,ticker_id,earnings_date,eps_estimate,reported_eps,surprise,ticker,strike_width,weekly_expiration,date
0,6,2,2024-06-12 20:00:00,10.84,10.96,0.0107,AVGO,5.0,1,2024-06-12
1,7,2,2024-03-07 21:00:00,10.29,10.99,0.0678,AVGO,5.0,1,2024-03-07
2,8,2,2023-12-07 21:00:00,10.98,11.06,0.0071,AVGO,5.0,1,2023-12-07
3,9,2,2023-08-31 20:00:00,10.42,10.54,0.0112,AVGO,5.0,1,2023-08-31
4,10,2,2023-06-01 20:00:00,10.08,10.32,0.0236,AVGO,5.0,1,2023-06-01
...,...,...,...,...,...,...,...,...,...,...
4331,6507,554,2023-08-08 11:00:00,1.32,1.41,0.0719,ZTS,NaN,0,2023-08-07
4332,6508,554,2023-05-04 11:00:00,1.26,1.31,0.0389,ZTS,NaN,0,2023-05-03
4333,6509,554,2023-02-14 12:00:00,1.05,1.15,0.0990,ZTS,NaN,0,2023-02-13
4334,6510,554,2022-11-03 11:00:00,1.24,1.21,-0.0207,ZTS,NaN,0,2022-11-02


In [6]:
result = df[['ticker', 'date']]
result.dtypes

ticker    object
date      object
dtype: object

In [7]:
# Convert date column to datetime if not already in datetime format
result['date'] = pd.to_datetime(result['date']).dt.date

def get_prices(ticker, date):
    # Create a Ticker object
    stock = yf.Ticker(ticker)
    
    # Convert date to string format for closing price
    start_date = date.strftime("%Y-%m-%d")
    end_date = (datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
    
    # Fetch historical data for closing price
    data = stock.history(start=start_date, end=end_date)
    
    # Check if any data is returned
    closing_price = None
    if not data.empty:
        # Extract the closing price
        closing_price = data['Close'].iloc[0]
    
    # Convert date to string format for next day's open price
    next_day = (datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
    next_end_date = (datetime.strptime(next_day, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
    
    # Fetch historical data for next day's open price
    next_day_data = stock.history(start=next_day, end=next_end_date)
    
    # Check if any data is returned
    open_price = None
    if not next_day_data.empty:
        # Extract the opening price
        open_price = next_day_data['Open'].iloc[0]
    
    return closing_price, open_price

# Initialize lists to store the prices
closing_prices = []
open_prices = []

# Iterate over each row in the result DataFrame
for index, row in result.iterrows():
    ticker = row['ticker']
    date = row['date']
    closing_price, open_price = get_prices(ticker, date)
    closing_prices.append(closing_price)
    open_prices.append(open_price)
    if closing_price:
        print(f"The closing price for {ticker} on {date} was {closing_price}")
    else:
        print(f"No data available for {ticker} on {date}")
    if open_price:
        print(f"The opening price for {ticker} on the next working day after {date} was {open_price}")
    else:
        print(f"No data available for {ticker} on the next working day after {date}")

# Add the closing prices and open prices as new columns in the result DataFrame
result['close_price'] = closing_prices
result['open_price'] = open_prices

The closing price for AVGO on 2024-06-12 was 149.55099487304688
The opening price for AVGO on the next working day after 2024-06-12 was 171.0
The closing price for AVGO on 2024-03-07 was 140.7010040283203
The opening price for AVGO on the next working day after 2024-03-07 was 139.4250030517578
The closing price for AVGO on 2023-12-07 was 92.22599792480469
The opening price for AVGO on the next working day after 2023-12-07 was 92.78399658203125
The closing price for AVGO on 2023-08-31 was 92.28900146484375
The opening price for AVGO on the next working day after 2023-08-31 was 90.18699645996094
The closing price for AVGO on 2023-06-01 was 78.99500274658203
The opening price for AVGO on the next working day after 2023-06-01 was 79.06400299072266
The closing price for AVGO on 2023-03-02 was 59.8650016784668
The opening price for AVGO on the next working day after 2023-03-02 was 61.77000045776367
The closing price for AVGO on 2022-12-08 was 53.108001708984375
The opening price for AVGO on 

The closing price for ENPH on 2022-07-26 was 216.10000610351562
The opening price for ENPH on the next working day after 2022-07-26 was 235.27999877929688
The closing price for HUBB on 2024-04-29 was 407.19000244140625
The opening price for HUBB on the next working day after 2024-04-29 was 394.9700012207031
The closing price for HUBB on 2024-01-29 was 332.55999755859375
The opening price for HUBB on the next working day after 2024-01-29 was 333.29998779296875
The closing price for HUBB on 2023-10-30 was 283.3299865722656
The opening price for HUBB on the next working day after 2023-10-30 was 265.20001220703125
The closing price for HUBB on 2023-07-24 was 333.7799987792969
The opening price for HUBB on the next working day after 2023-07-24 was 330.0
The closing price for HUBB on 2023-04-24 was 232.38999938964844
The opening price for HUBB on the next working day after 2023-04-24 was 251.9199981689453
The closing price for HUBB on 2023-01-30 was 229.88999938964844
The opening price for H

The closing price for J on 2023-08-07 was 128.49000549316406
The opening price for J on the next working day after 2023-08-07 was 127.62000274658203
The closing price for J on 2023-05-08 was 117.29000091552734
The opening price for J on the next working day after 2023-05-08 was 116.08999633789062


J: No price data found, symbol may be delisted (1d 2022-11-20 -> 2022-11-21)


The closing price for J on 2023-02-06 was 124.91999816894531
The opening price for J on the next working day after 2023-02-06 was 124.93000030517578


...

WDC: No price data found, symbol may be delisted (1d 2023-10-29 -> 2023-10-30)


The closing price for WDC on 2024-01-25 was 60.33000183105469
The opening price for WDC on the next working day after 2024-01-25 was 58.86000061035156
No data available for WDC on 2023-10-29
The opening price for WDC on the next working day after 2023-10-29 was 43.150001525878906
The closing price for WDC on 2023-07-31 was 42.560001373291016
The opening price for WDC on the next working day after 2023-07-31 was 42.0
The closing price for WDC on 2023-05-08 was 34.18000030517578
The opening price for WDC on the next working day after 2023-05-08 was 33.209999084472656
The closing price for WDC on 2023-01-31 was 43.95000076293945
The opening price for WDC on the next working day after 2023-01-31 was 43.84000015258789
The closing price for WDC on 2022-10-26 was 35.279998779296875
The opening price for WDC on the next working day after 2022-10-26 was 36.88999938964844
The closing price for WDC on 2022-08-04 was 49.90999984741211
The opening price for WDC on the next working day after 2022-08

WMB: No price data found, symbol may be delisted (1d 2023-02-20 -> 2023-02-21)


The closing price for WMB on 2023-05-03 was 28.959999084472656
The opening price for WMB on the next working day after 2023-05-03 was 29.260000228881836
No data available for WMB on 2023-02-20
The opening price for WMB on the next working day after 2023-02-20 was 31.1299991607666
The closing price for WMB on 2022-10-31 was 32.72999954223633
The opening price for WMB on the next working day after 2022-10-31 was 33.279998779296875
The closing price for WMB on 2022-08-01 was 34.0
The opening price for WMB on the next working day after 2022-08-01 was 34.150001525878906
The closing price for WTW on 2024-04-24 was 264.4800109863281
The opening price for WTW on the next working day after 2024-04-24 was 251.2100067138672
The closing price for WTW on 2024-02-05 was 250.4600067138672
The opening price for WTW on the next working day after 2024-02-05 was 256.5899963378906
The closing price for WTW on 2023-10-25 was 207.74000549316406
The opening price for WTW on the next working day after 2023-10

The closing price for ZBH on 2023-05-01 was 138.8300018310547
The opening price for ZBH on the next working day after 2023-05-01 was 146.0
The closing price for ZBH on 2023-02-02 was 127.41000366210938
The opening price for ZBH on the next working day after 2023-02-02 was 127.61000061035156
The closing price for ZBH on 2022-11-01 was 111.83999633789062
The opening price for ZBH on the next working day after 2022-11-01 was 114.98999786376953
The closing price for ZBH on 2022-08-01 was 109.76000213623047
The opening price for ZBH on the next working day after 2022-08-01 was 113.01000213623047
The closing price for ZTS on 2024-05-01 was 158.5
The opening price for ZTS on the next working day after 2024-05-01 was 163.10000610351562
The closing price for ZTS on 2024-02-12 was 196.69000244140625
The opening price for ZTS on the next working day after 2024-02-12 was 186.60000610351562
The closing price for ZTS on 2023-11-01 was 151.44000244140625
The opening price for ZTS on the next working 

In [9]:
result

,ticker,date,close_price,open_price
0,AVGO,2024-06-12,149.550995,171.000000
1,AVGO,2024-03-07,140.701004,139.425003
2,AVGO,2023-12-07,92.225998,92.783997
3,AVGO,2023-08-31,92.289001,90.186996
4,AVGO,2023-06-01,78.995003,79.064003
...,...,...,...,...
4331,ZTS,2023-08-07,180.690002,182.149994
4332,ZTS,2023-05-03,177.729996,168.000000
4333,ZTS,2023-02-13,163.139999,173.440002
4334,ZTS,2022-11-02,147.360001,134.149994


In [19]:
result['gap'] = result['open_price']-result['close_price']
result['perc'] = result['gap'] / result['close_price']*100
result['perc'] = round(result['perc'],2)
result['gap'] = result['gap'].abs()

In [20]:
result

,ticker,date,close_price,open_price,gap,perc
0,AVGO,2024-06-12,149.550995,171.000000,21.449005,14.34
1,AVGO,2024-03-07,140.701004,139.425003,1.276001,-0.91
2,AVGO,2023-12-07,92.225998,92.783997,0.557999,0.61
3,AVGO,2023-08-31,92.289001,90.186996,2.102005,-2.28
4,AVGO,2023-06-01,78.995003,79.064003,0.069000,0.09
...,...,...,...,...,...,...
4331,ZTS,2023-08-07,180.690002,182.149994,1.459991,0.81
4332,ZTS,2023-05-03,177.729996,168.000000,9.729996,-5.47
4333,ZTS,2023-02-13,163.139999,173.440002,10.300003,6.31
4334,ZTS,2022-11-02,147.360001,134.149994,13.210007,-8.96


In [22]:
# Merge the result DataFrame with the primary df DataFrame to fill in the additional columns
merged_result = result.merge(df, on=['ticker', 'date'], how='left')

# Ensure the additional columns are updated and no duplication of columns occurs
merged_result = merged_result.loc[:, ~merged_result.columns.duplicated()]

In [23]:
merged_result

,ticker,date,close_price,open_price,gap,perc,id,ticker_id,earnings_date,eps_estimate,reported_eps,surprise,strike_width,weekly_expiration
0,AVGO,2024-06-12,149.550995,171.000000,21.449005,14.34,6,2,2024-06-12 20:00:00,10.84,10.96,0.0107,5.0,1
1,AVGO,2024-03-07,140.701004,139.425003,1.276001,-0.91,7,2,2024-03-07 21:00:00,10.29,10.99,0.0678,5.0,1
2,AVGO,2023-12-07,92.225998,92.783997,0.557999,0.61,8,2,2023-12-07 21:00:00,10.98,11.06,0.0071,5.0,1
3,AVGO,2023-08-31,92.289001,90.186996,2.102005,-2.28,9,2,2023-08-31 20:00:00,10.42,10.54,0.0112,5.0,1
4,AVGO,2023-06-01,78.995003,79.064003,0.069000,0.09,10,2,2023-06-01 20:00:00,10.08,10.32,0.0236,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,ZTS,2023-08-07,180.690002,182.149994,1.459991,0.81,6507,554,2023-08-08 11:00:00,1.32,1.41,0.0719,NaN,0
4336,ZTS,2023-05-03,177.729996,168.000000,9.729996,-5.47,6508,554,2023-05-04 11:00:00,1.26,1.31,0.0389,NaN,0
4337,ZTS,2023-02-13,163.139999,173.440002,10.300003,6.31,6509,554,2023-02-14 12:00:00,1.05,1.15,0.0990,NaN,0
4338,ZTS,2022-11-02,147.360001,134.149994,13.210007,-8.96,6510,554,2022-11-03 11:00:00,1.24,1.21,-0.0207,NaN,0


In [35]:
merged_result['escalones_gap'] = merged_result['gap'] / merged_result['strike_width']
merged_result['gap'] = round(merged_result['gap'],2)
merged_result['escalones_gap'] = round(merged_result['escalones_gap'],1)
merged_result = merged_result.drop(columns=['id'])

In [42]:
merged_result

,ticker,date,close_price,open_price,gap,perc,ticker_id,earnings_date,eps_estimate,reported_eps,surprise,strike_width,weekly_expiration,escalones_gap
0,AVGO,2024-06-12,149.550995,171.000000,21.45,14.34,2,2024-06-12 20:00:00,10.84,10.96,0.0107,5.0,1,4.3
1,AVGO,2024-03-07,140.701004,139.425003,1.28,-0.91,2,2024-03-07 21:00:00,10.29,10.99,0.0678,5.0,1,0.3
2,AVGO,2023-12-07,92.225998,92.783997,0.56,0.61,2,2023-12-07 21:00:00,10.98,11.06,0.0071,5.0,1,0.1
3,AVGO,2023-08-31,92.289001,90.186996,2.10,-2.28,2,2023-08-31 20:00:00,10.42,10.54,0.0112,5.0,1,0.4
4,AVGO,2023-06-01,78.995003,79.064003,0.07,0.09,2,2023-06-01 20:00:00,10.08,10.32,0.0236,5.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,ZTS,2023-08-07,180.690002,182.149994,1.46,0.81,554,2023-08-08 11:00:00,1.32,1.41,0.0719,NaN,0,NaN
4336,ZTS,2023-05-03,177.729996,168.000000,9.73,-5.47,554,2023-05-04 11:00:00,1.26,1.31,0.0389,NaN,0,NaN
4337,ZTS,2023-02-13,163.139999,173.440002,10.30,6.31,554,2023-02-14 12:00:00,1.05,1.15,0.0990,NaN,0,NaN
4338,ZTS,2022-11-02,147.360001,134.149994,13.21,-8.96,554,2022-11-03 11:00:00,1.24,1.21,-0.0207,NaN,0,NaN


In [37]:
merged_result.to_csv('open_close_gap_earnings', index=False)

ESTUDIO DE LA VARIANZA POR ACCIÓN

In [43]:
# Agrupar por 'ticker' y calcular estadísticas
summary_stats = merged_result.groupby('ticker')['escalones_gap'].agg(
    mean_gap='mean',
    std_dev_gap='std',
    min_gap='min',
    max_gap='max',
    count='count'
).reset_index()

summary_stats

,ticker,mean_gap,std_dev_gap,min_gap,max_gap,count
0,A,1.9625,1.204678,0.7,4.0,8
1,AAL,0.6375,0.453360,0.0,1.3,8
2,AAP,11.6625,12.928146,0.2,33.0,8
3,AAPL,1.1250,0.688165,0.6,2.7,8
4,ABBV,0.8750,0.565054,0.0,1.6,8
...,...,...,...,...,...,...
541,XYL,NaN,NaN,NaN,NaN,0
542,YUM,NaN,NaN,NaN,NaN,0
543,ZBH,NaN,NaN,NaN,NaN,0
544,ZBRA,NaN,NaN,NaN,NaN,0


In [45]:
# Agrupar por 'ticker' y calcular estadísticas
summary_stats = merged_result.groupby('ticker')['escalones_gap'].agg(
    mean_gap='mean',
    std_dev_gap='std',
    var_gap='var',
    min_gap='min',
    max_gap='max',
    count='count'
).reset_index()

# Añadir la columna 'escalones_gap' original para cada ticker
escalones_gap_original = merged_result.groupby('ticker')['escalones_gap'].apply(list).reset_index()
escalones_gap_original.columns = ['ticker', 'escalones_gap_list']

# Combinar los resultados estadísticos con los valores originales de 'escalones_gap'
summary_stats = pd.merge(summary_stats, escalones_gap_original, on='ticker')


In [50]:
summary_stats = summary_stats.sort_values(by='mean_gap', ascending=False).reset_index(drop=True)

summary_stats.head(30)

,ticker,mean_gap,std_dev_gap,var_gap,min_gap,max_gap,count,escalones_gap_list
0,BKNG,24.275000,22.399729,501.747857,0.7,63.0,8,"[45.4, 40.7, 11.7, 63.0, 5.4, 0.7, 14.2, 13.1]"
1,SEDG,22.850000,12.795870,163.734286,4.0,36.4,8,"[4.0, 14.4, 11.9, 35.0, 36.1, 15.7, 29.3, 36.4]"
2,ENPH,19.450000,11.494844,132.131429,0.9,42.0,8,"[0.9, 19.0, 14.0, 16.8, 42.0, 18.4, 25.3, 19.2]"
3,EL,16.414286,16.138817,260.461429,5.5,51.1,7,"[6.3, nan, 19.6, 10.1, 51.1, 7.1, 15.2, 5.5]"
4,ALGN,14.750000,8.214099,67.471429,3.2,25.6,8,"[4.5, 14.3, 22.7, 25.6, 13.0, 21.6, 13.1, 3.2]"
5,DG,11.925000,9.483331,89.933571,2.8,29.0,8,"[4.6, 8.0, 5.3, 29.0, 22.2, 2.8, 16.0, 7.5]"
6,AAP,11.662500,12.928146,167.136964,0.2,33.0,8,"[5.9, 1.0, 3.6, 0.2, 33.0, 4.4, 28.9, 16.3]"
7,EXPE,11.225000,8.047670,64.765000,4.8,28.9,8,"[13.8, 28.9, 13.2, 11.7, 5.2, 6.3, 5.9, 4.8]"
8,DLTR,10.712500,8.181938,66.944107,1.3,20.6,8,"[2.3, 20.5, 1.3, 11.2, 20.6, 2.0, 10.8, 17.0]"
9,DELL,8.962500,12.077954,145.876964,0.0,30.0,8,"[26.1, 30.0, 4.0, 7.8, 0.0, 1.1, 0.7, 2.0]"


In [52]:
# Añadir la columna 'escalones_gap' original para cada ticker
escalones_gap_original = merged_result.groupby('ticker')['escalones_gap'].apply(list).reset_index()
escalones_gap_original.columns = ['ticker', 'escalones_gap_list']

# Añadir la columna 'strike_width' para cada ticker (asumiendo que cada ticker tiene un valor único de 'strike_width')
strike_width = df.groupby('ticker')['strike_width'].first().reset_index()

# Combinar los resultados estadísticos con los valores originales de 'escalones_gap' y 'strike_width'
summary_stats = pd.merge(summary_stats, escalones_gap_original, on='ticker')
summary_stats = pd.merge(summary_stats, strike_width, on='ticker')

# Redondear los valores a 2 decimales
summary_stats = summary_stats.round(2)

In [53]:
summary_stats

,ticker,mean_gap,std_dev_gap,var_gap,min_gap,max_gap,count,escalones_gap_list_x,escalones_gap_list_y,strike_width
0,BKNG,24.28,22.40,501.75,0.7,63.0,8,"[45.4, 40.7, 11.7, 63.0, 5.4, 0.7, 14.2, 13.1]","[45.4, 40.7, 11.7, 63.0, 5.4, 0.7, 14.2, 13.1]",5.0
1,SEDG,22.85,12.80,163.73,4.0,36.4,8,"[4.0, 14.4, 11.9, 35.0, 36.1, 15.7, 29.3, 36.4]","[4.0, 14.4, 11.9, 35.0, 36.1, 15.7, 29.3, 36.4]",1.0
2,ENPH,19.45,11.49,132.13,0.9,42.0,8,"[0.9, 19.0, 14.0, 16.8, 42.0, 18.4, 25.3, 19.2]","[0.9, 19.0, 14.0, 16.8, 42.0, 18.4, 25.3, 19.2]",1.0
3,EL,16.41,16.14,260.46,5.5,51.1,7,"[6.3, nan, 19.6, 10.1, 51.1, 7.1, 15.2, 5.5]","[6.3, nan, 19.6, 10.1, 51.1, 7.1, 15.2, 5.5]",1.0
4,ALGN,14.75,8.21,67.47,3.2,25.6,8,"[4.5, 14.3, 22.7, 25.6, 13.0, 21.6, 13.1, 3.2]","[4.5, 14.3, 22.7, 25.6, 13.0, 21.6, 13.1, 3.2]",2.5
...,...,...,...,...,...,...,...,...,...,...
541,XYL,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan]",NaN
542,YUM,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan]",NaN
543,ZBH,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan]",NaN
544,ZBRA,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan]",NaN


In [56]:
summary_stats = summary_stats.drop(columns=['escalones_gap_list_y'])
summary_stats = summary_stats.rename(columns={'escalones_gap_list_x': 'escalones_unit'})

In [57]:
summary_stats

,ticker,mean_gap,std_dev_gap,var_gap,min_gap,max_gap,count,escalones_unit,strike_width
0,BKNG,24.28,22.40,501.75,0.7,63.0,8,"[45.4, 40.7, 11.7, 63.0, 5.4, 0.7, 14.2, 13.1]",5.0
1,SEDG,22.85,12.80,163.73,4.0,36.4,8,"[4.0, 14.4, 11.9, 35.0, 36.1, 15.7, 29.3, 36.4]",1.0
2,ENPH,19.45,11.49,132.13,0.9,42.0,8,"[0.9, 19.0, 14.0, 16.8, 42.0, 18.4, 25.3, 19.2]",1.0
3,EL,16.41,16.14,260.46,5.5,51.1,7,"[6.3, nan, 19.6, 10.1, 51.1, 7.1, 15.2, 5.5]",1.0
4,ALGN,14.75,8.21,67.47,3.2,25.6,8,"[4.5, 14.3, 22.7, 25.6, 13.0, 21.6, 13.1, 3.2]",2.5
...,...,...,...,...,...,...,...,...,...
541,XYL,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]",NaN
542,YUM,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]",NaN
543,ZBH,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]",NaN
544,ZBRA,NaN,NaN,NaN,NaN,NaN,0,"[nan, nan, nan, nan, nan, nan, nan, nan]",NaN


In [58]:
summary_stats.to_csv('volatile_top_tickers', index=False)